In [1]:
from mads_datasets import DatasetFactoryProvider, DatasetType
from mltrainer.preprocessors import PaddedPreprocessor
preprocessor = PaddedPreprocessor()

gesturesdatasetfactory = DatasetFactoryProvider.create_factory(DatasetType.GESTURES)
streamers = gesturesdatasetfactory.create_datastreamer(batchsize=32, preprocessor=preprocessor)
train = streamers["train"]
valid = streamers["valid"]

2024-12-05 18:23:08.674 | INFO     | mads_datasets.base:download_data:121 - Folder already exists at /home/azureuser/.cache/mads_datasets/gestures
100%|██████████| 651/651 [00:05<00:00, 130.15it/s]


In [2]:
from mltrainer import TrainerSettings, ReportTypes
from mltrainer.metrics import Accuracy
from pathlib import Path

accuracy = Accuracy()

settings = TrainerSettings(
    epochs=20,
    metrics=[accuracy],
    logdir=Path("gestures_gru"),
    train_steps=len(train),
    valid_steps=len(valid),
    reporttypes=[ReportTypes.GIN, ReportTypes.TENSORBOARD, ReportTypes.MLFLOW],
    scheduler_kwargs={"factor": 0.5, "patience": 5},
    earlystop_kwargs=None
)
settings

2024-12-05 18:23:40.787 | INFO     | mltrainer.settings:check_path:61 - Created logdir /home/azureuser/MachineLearning/notebooks/3_recurrent_networks/gestures_gru


epochs: 20
metrics: [Accuracy]
logdir: gestures_gru
train_steps: 81
valid_steps: 20
reporttypes: [<ReportTypes.GIN: 1>, <ReportTypes.TENSORBOARD: 2>, <ReportTypes.MLFLOW: 3>]
optimizer_kwargs: {'lr': 0.001, 'weight_decay': 1e-05}
scheduler_kwargs: {'factor': 0.5, 'patience': 5}
earlystop_kwargs: None

In [4]:
len(train), len(valid)

(81, 20)

In [5]:
import gin
from mltrainer import rnn_models, Trainer

# Laad de configuratie
gin.parse_config_file("gestures_gru.gin")

# Voorbeeld van het gebruik van de configuratie
model = rnn_models.GRUmodel()

# Voorbeeld van het gebruik van trainstreamer en validstreamer
trainstreamer = train.stream()
validstreamer = valid.stream()
x, y = next(iter(trainstreamer))
x.shape, y

/home/azureuser/MachineLearning/.venv/lib/python3.11/site-packages/gin/config.py:615: FutureWarning: `NLLLoss2d` has been deprecated. Please use `NLLLoss` instead as a drop-in replacement and see https://pytorch.org/docs/main/nn.html#torch.nn.NLLLoss for more details.
  decorated_class = decorating_meta(cls.__name__, (cls,), overrides)


(torch.Size([32, 32, 3]),
 tensor([ 8,  4,  1,  4, 14, 18,  7, 16, 19,  2,  6, 13, 19, 15, 13,  5,  0,  6,
          9, 18, 19, 12, 11,  3,  6, 11, 13,  6, 16, 16, 12, 19]))

In [6]:
gin.get_bindings("GRUmodel")["config"]

{'input_size': 3,
 'hidden_size': 16,
 'dropout': 0.5,
 'num_layers': 2,
 'output_size': 20}

Device:

In [7]:
import torch
if torch.backends.mps.is_available() and torch.backends.mps.is_built():
    device = torch.device("mps")
    print("Using MPS")
elif torch.cuda.is_available():
    device = "cuda:0"
    print("using cuda")
else:
    device = "cpu"
    print("using cpu")

# on my mac, at least for the BaseRNN model, mps does not speed up training
# probably because the overhead of copying the data to the GPU is too high
# however, it might speed up training for larger models, with more parameters
device = "cpu"

using cpu


In [8]:
import mlflow
from datetime import datetime
from torch import optim

loss_fn = torch.nn.CrossEntropyLoss()

mlflow.set_tracking_uri("sqlite:///mlflow.db")
mlflow.set_experiment("gestures_gru")
modeldir = Path("../../models/gestures_gru/").resolve()
if not modeldir.exists():
    modeldir.mkdir(parents=True)

gin.parse_config_file("gestures_gru.gin")

with mlflow.start_run():
    mlflow.set_tag("model", "GRUmodel")
    mlflow.set_tag("dev", "raoul")
    mlflow.log_params(gin.get_bindings("GRUmodel")["config"])

    model = rnn_models.GRUmodel()

    trainer = Trainer(
        model=model,
        settings=settings,
        loss_fn=loss_fn,
        optimizer=optim.Adam,
        traindataloader=trainstreamer,
        validdataloader=validstreamer,
        scheduler=optim.lr_scheduler.ReduceLROnPlateau,
        device=device,
    )
    trainer.loop()

    tag = datetime.now().strftime("%Y%m%d-%H%M")
    modelpath = modeldir / (tag + "model.pt")
    torch.save(model, modelpath)

2024/12/05 18:29:41 INFO mlflow.tracking.fluent: Experiment with name 'gestures_gru' does not exist. Creating a new experiment.
2024-12-05 18:29:41.454 | INFO     | mltrainer.trainer:dir_add_timestamp:29 - Logging to gestures_gru/20241205-182941
100%|██████████| 81/81 [00:01<00:00, 44.19it/s]
2024-12-05 18:29:46.886 | INFO     | mltrainer.trainer:report:191 - Epoch 0 train 2.9990 test 2.9722 metric ['0.0844']
100%|██████████| 81/81 [00:01<00:00, 58.84it/s]
2024-12-05 18:29:48.438 | INFO     | mltrainer.trainer:report:191 - Epoch 1 train 2.9264 test 2.7787 metric ['0.1203']
100%|██████████| 81/81 [00:01<00:00, 56.80it/s]
2024-12-05 18:29:50.088 | INFO     | mltrainer.trainer:report:191 - Epoch 2 train 2.5260 test 2.3118 metric ['0.2125']
100%|██████████| 81/81 [00:01<00:00, 57.70it/s]
2024-12-05 18:29:51.676 | INFO     | mltrainer.trainer:report:191 - Epoch 3 train 2.2124 test 2.1383 metric ['0.2562']
100%|██████████| 81/81 [00:01<00:00, 57.75it/s]
2024-12-05 18:29:53.281 | INFO     | m

In [ ]:
model

In [ ]:
settings

In [ ]:
mlflow.end_run()